In [1]:
import pandas as pd 
import os 
import sys

sys.path.insert(0, os.getcwd() + '/code')
from viz import animate_play
from helper_functions import compute_orientation, face2face

Path:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week1.csv
Filename:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week1.csv
Path:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/players.csv
Filename:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/players.csv
Path:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week4.csv
Filename:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week4.csv
Path:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week2.csv
Filename:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week2.csv
Path:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/games.csv
Filename:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/games.csv
Path:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week5.csv
Filename:  /home/aschortgen/Documents/Divers/NFL-Big-Data-Bowl-2023/data/week5.csv
Path:  /home/aschortgen/

In [2]:
df_game = pd.read_csv("data/games.csv")
df_tracking = pd.read_csv("data/week1.csv")
df_play = pd.read_csv("data/plays.csv")
df_players = pd.read_csv("data/players.csv")
df_pffScoutingData = pd.read_csv("data/pffScoutingData.csv")

In [3]:
df_tracking = compute_orientation(df_tracking)
print(df_tracking.head())

<bound method NDFrame.head of              gameId  playId    nflId  frameId                     time  \
0        2021090900      97  25511.0        1  2021-09-10T00:26:31.100   
1        2021090900      97  25511.0        2  2021-09-10T00:26:31.200   
2        2021090900      97  25511.0        3  2021-09-10T00:26:31.300   
3        2021090900      97  25511.0        4  2021-09-10T00:26:31.400   
4        2021090900      97  25511.0        5  2021-09-10T00:26:31.500   
...             ...     ...      ...      ...                      ...   
1118117  2021091300    4845      NaN       30  2021-09-14T03:54:20.600   
1118118  2021091300    4845      NaN       31  2021-09-14T03:54:20.700   
1118119  2021091300    4845      NaN       32  2021-09-14T03:54:20.800   
1118120  2021091300    4845      NaN       33  2021-09-14T03:54:20.900   
1118121  2021091300    4845      NaN       34  2021-09-14T03:54:21.000   

         jerseyNumber      team playDirection      x      y      s     a  \
0    

In [4]:
animate_play(df_tracking,df_play,df_players,df_pffScoutingData,2021090900,97).show()